In [1]:
import numpy as np
import pandas as pd
import math
import sys, os
#import seaborn as sns
#from scipy import stats
#from pathlib import Path
#import matplotlib.pyplot as plt
#from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.datasets import fetch_california_housing
#import math
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import optuna
from optuna.samplers import TPESampler
import lightgbm as lgbm
#from xgboost import XGBRegressor
#import xgboost as xgb
#from catboost import CatBoostRegressor
#from lightgbm.sklearn import LGBMRegressor
#from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import mplcyberpunk
import seaborn as sns

In [2]:
train_time = 1 * 60 * 60
objective = 'lgbmregressor'

In [3]:
sys.path.append(os.path.realpath('..'))

In [4]:
train_df = pd.read_csv(r"..\data\train.csv", index_col=0)
test_df = pd.read_csv(r"..\data\test.csv", index_col=0)
sample_sub = pd.read_csv(r"..\data\sample_submission.csv", index_col=0)

In [5]:
features = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
target = ['Class']

In [6]:
np.unique(train_df[target], return_counts=True)

(array([0, 1], dtype=int64), array([218660,    469], dtype=int64))

In [7]:
train_df[features] = train_df[features].apply(lambda iterator: ((iterator.max() - iterator)/(iterator.max() - iterator.min())).round(2))
test_df[features] = test_df[features].apply(lambda iterator: ((iterator.max() - iterator)/(iterator.max() - iterator.min())).round(2))

In [8]:
np.unique(train_df[target], return_counts=True)

(array([0, 1], dtype=int64), array([218660,    469], dtype=int64))

In [9]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [17]:
RANDOM_STATE = 12 
FOLDS = 5
param_grid_history = {}
train_df_score_keeper = train_df.copy()

train_df_score_keeper['record_weight'] = 0

class callbacker():
    def __init__(self):
        self.totals = []
        
    def add_totals(self, payload):
        #print(payload.evaluation_result_list[0][2])
        self.totals.append(payload.evaluation_result_list[0][2])
        
    def get_totals(self):
        return self.totals
    
    def get_mean(self):
        return sum(np.array(self.totals)) / len(self.totals)
    
best_auc = 0

def objective_v3(trial):
    global best_auc
    oof = np.zeros(len(X))
    scores = []
    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.9),
        'max_depth': trial.suggest_int('max_depth', 3, 8), 
        #'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 100),
        #'num_leaves': trial.suggest_int('num_leaves', 100, 10000),
        #'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        #'lambda_l1': trial.suggest_float('lambda_l1', 0.1, 1),
        #'lambda_l2': trial.suggest_float('lambda_l2', 0.1, 1),
        #'subsample': trial.suggest_float('subsample', 0.1, 1),
        #'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.1, 5),
        #'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1),
        #'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1),
        #'subsample_freq': trial.suggest_int('subsample_freq', 1, 10), 
        #'min_child_samples': trial.suggest_int('min_child_samples', 10, 1000), 
        'reg_lambda': trial.suggest_int('reg_lambda', 1, 5), 
        #'n_estimators': trial.suggest_int('n_estimators', 10, 100000),
        'objective':'binary',
    } 
    reg = lgbm.LGBMRegressor(**param_grid, metric='AUC', random_state=1)
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
    for z in range(10):
        for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
            cber = callbacker()
            print(10*"=", f"Fold={fold+1}", 100*"=")
            X_train, X_valid = train_df.iloc[train_idx][features], train_df.iloc[valid_idx][features]
            y_train , y_valid = train_df[target].iloc[train_idx] , train_df[target].iloc[valid_idx] 

            reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[lgbm.early_stopping(100, verbose=True), cber.add_totals])

            train_df_score_keeper.loc[train_idx, 'record_weight'] = (train_df_score_keeper.loc[train_idx, 'record_weight'] + cber.get_mean()) / 2

            oof[valid_idx] = reg.predict(X_valid)

        roc_auc = roc_auc_score(Y, oof)

        if roc_auc > best_auc:
            best_auc = roc_auc
            train_final_preds = reg.predict(train_df[features])
            test_final_preds = reg.predict(test_df[features])
            train_submission = pd.DataFrame(data={'id': train_df.index, target[0]: train_final_preds})
            train_submission.to_csv(fr'rendered_data/{objective}_train_submission.csv', index=False)
            test_submission = pd.DataFrame(data={'id': test_df.index, target[0]: test_final_preds})
            test_submission.to_csv(fr'rendered_data/{objective}_test_submission.csv', index=False)
        
    
    param_grid_history[roc_auc] = param_grid
    train_df_score_keeper.to_csv('1000kfoldwithweights.csv')

    return roc_auc

In [18]:
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name=objective)
study.optimize(objective_v3, n_trials=1000)

[I 2023-01-28 12:32:30,322] A new study created in memory with name: lgbmregressor


========== Fold=1 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[11]	valid_0's auc: 0.811072
========== Fold=2 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[13]	valid_0's auc: 0.798009
========== Fold=3 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[28]	valid_0's auc: 0.781444
========== Fold=4 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration i

[I 2023-01-28 12:32:57,815] Trial 0 finished with value: 0.7966428197957827 and parameters: {'learning_rate': 0.49315376716500814, 'max_depth': 3, 'reg_lambda': 4}. Best is trial 0 with value: 0.7966428197957827.


========== Fold=1 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1]	valid_0's auc: 0.757511
========== Fold=2 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1]	valid_0's auc: 0.754247
========== Fold=3 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[3]	valid_0's auc: 0.706074
========== Fold=4 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:


[I 2023-01-28 12:33:32,362] Trial 1 finished with value: 0.7335791495671347 and parameters: {'learning_rate': 0.8526495325908112, 'max_depth': 8, 'reg_lambda': 3}. Best is trial 0 with value: 0.7966428197957827.


...

[I 2023-01-28 12:59:23,645] Trial 50 finished with value: 0.8120583464665669 and parameters: {'learning_rate': 0.22835105624762847, 'max_depth': 5, 'reg_lambda': 4}. Best is trial 44 with value: 0.8256774154732343.


========== Fold=1 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[39]	valid_0's auc: 0.832468
========== Fold=2 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[57]	valid_0's auc: 0.821846
========== Fold=3 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.773128
========== Fold=4 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration 

[I 2023-01-28 12:59:55,873] Trial 51 finished with value: 0.8136713500353091 and parameters: {'learning_rate': 0.07347028044840785, 'max_depth': 5, 'reg_lambda': 5}. Best is trial 44 with value: 0.8256774154732343.


========== Fold=1 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[36]	valid_0's auc: 0.828335
========== Fold=2 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[59]	valid_0's auc: 0.836589
========== Fold=3 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[95]	valid_0's auc: 0.764085
========== Fold=4 ====================================================================================================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration i

[W 2023-01-28 12:59:59,278] Trial 52 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "D:\source\repos\venv\Python310\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_42276\2229885829.py", line 55, in objective_v3
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[lgbm.early_stopping(100, verbose=True), cber.add_totals])
  File "D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py", line 895, in fit
    super().fit(X, y, sample_weight=sample_weight, init_score=init_score,
  File "D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py", line 748, in fit
    self._Booster = train(
  File "D:\source\repos\venv\Python310\lib\site-packages\lightgbm\engine.py", line 292, in train
    booster.update(fobj=fobj)
  File "D:\source\repos\venv\Python310\lib\site-packages\lightgbm\basic.py", 

In [ ]:
train_df_score_keeper['record_weight'].describe()

In [ ]:
train_df_score_keeper['record_weight'][-200:-100]

In [ ]:
train_df_score_keeper.to_csv('1000kfoldwithweights.csv')

In [ ]:
#train_df = train_df_score_keeper[train_df_score_keeper['record_weight'] > train_df_score_keeper['record_weight'].describe()['25%']].copy()
#train_df.reset_index(inplace=True)

In [ ]:
train_df

In [ ]:
parm_grid_bytes = pickle.dumps(param_grid_history)
with open(f"rendered_data/{objective}_bytes.hex", "wb") as binary_file:
    binary_file.write(parm_grid_bytes)

In [ ]:
def train(param_grid):
    reg = lgbm.LGBMRegressor(**param_grid, metric='AUC', random_state=1)
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.20)
    
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[lgbm.early_stopping(100, verbose=True)], verbose=False)
    
    return reg

percent = math.ceil(len(param_grid_history.keys()) * .1)
top = sorted(list(param_grid_history.keys()))[-1:]

train_preds = []
test_preds = []
for key in tqdm(top):
    model = train(param_grid_history[key])
    train_preds.append(model.predict(train_df[features]))
    test_preds.append(model.predict(test_df[features]))

train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

In [ ]:
plt.style.use("cyberpunk")

histplot_hyperparams = {
        'alpha':0.4,
        'stat':'percent',
        'bins':100
    }

n_cols = 1
n_rows = 1
cell_with_dim = 6
cell_height_dim = 3

fig, ax = plt.subplots(n_rows, n_cols, figsize=(n_cols * cell_with_dim, n_rows * cell_height_dim))

sns.histplot(train_final_preds, ax=ax, **histplot_hyperparams)
#sns.histplot(train_df[target], ax=ax, **histplot_hyperparams)

mplcyberpunk.make_lines_glow(ax)
print(np.unique(train_final_preds, return_counts=True))
print(roc_auc_score(train_df[target], train_final_preds))
plt.show()

In [ ]:
train_submission = pd.DataFrame(data={'id': train_df.index, target[0]: train_final_preds})
train_submission.to_csv(fr'rendered_data/{objective}_train_submission.csv', index=False)

test_submission = pd.DataFrame(data={'id': test_df.index, target[0]: test_final_preds})
test_submission.to_csv(fr'rendered_data/{objective}_test_submission.csv', index=False)